In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from math import gcd, log2, pi
from fractions import Fraction
from random import randint

In [2]:
# Define a function to perform Quantum Fourier Transform
def qft(circuit, qubits):
    n= len(qubits)

    # Perform Hadamard and controlled phase rotations
    for i in range(n):
        circuit.h(qubits[i])
        for k in range(i+1, n):
            circuit.cp(pi/(2**(k - i)), qubits[k], qubits[i])

    # Reverse the order of the qubits
    for i in range(n//2):
        circuit.swap(qubits[i], qubits[n - i - 1])

In [3]:
# Define a function to perform Inverse Quantum Fourier Transform
def iqft(circuit, qubits):
    n= len(qubits)

    # Perform the inverse of the operations applied during the Quantum Fourier Transform (in reverse order)
    for i in range(n//2):
        circuit.swap(qubits[i], qubits[n - i - 1])

    for i in reversed(range(n)):
        for k in reversed(range(i)):
            circuit.cp(-pi/(2**(i - k)), qubits[k], qubits[i])
        circuit.h(qubits[i])

In [4]:
# Define a function to perform phase estimation for finding the order of a mod N in Shor's Algorithm
def phase_estimation(circuit, control, target, a, N):
    n= len(control)

    circuit.h(control)
    circuit.x(target[0])

    mod_powers = [(a**(2**j))%N for j in range(n)] # precompute a^(2^j) mod N for all control qubits

    # Apply 'simulated' controlled modular multiplications
    for j in range(n):
        i= int(log2(mod_powers[j]))
        circuit.cx(control[j], target[i])

    iqft(circuit, control)

In [5]:
# Factor 33 using Shor's Algorithm
flag= False

while flag is False: # repeat until non- trivial factors are found
    a= randint(2, 32)
    if gcd(a, 33)!= 1:
        continue

    control= QuantumRegister(12) # 12 qubits for a high precision
    target= QuantumRegister(6) # 6 qubits to represent all numbers from 0 to 32
    c= ClassicalRegister(12)
    circuit= QuantumCircuit(control, target, c)

    phase_estimation(circuit, control, target, a, 33)
    circuit.measure(control, c)

    counts= AerSimulator().run(circuit, shots=1000).result().get_counts()
    measured= max(counts, key=counts.get)
    phase= int(measured, 2)/(2**12) # convert the binary string to a phase value
    r= Fraction(phase).limit_denominator(33).denominator # use continued fractions to estimate r from the measured phase

    if r%2== 1: # retry if r is odd
        continue

    p= gcd(pow(a, r//2)- 1, 33)
    q= gcd(pow(a, r//2)+ 1, 33)

    if p== 1 or q== 1 or p== 33 or q== 33: # retry if the factors are non- trivial
        continue

    print(f"The Prime Factors of 33 are {p} and {q}")
    flag= True


The Prime Factors of 33 are 11 and 3
